
```yaml
version: '3.8'
services:
  Flaskapp: # This is where we can assign a hostname
    build: .
    ports:
      - "5000:5000"
    depends_on:
      - minio
      - postgres
      - redis

  minio:
    image: minio/minio
    environment:
      MINIO_ACCESS_KEY: minio
      MINIO_SECRET_KEY: minio123
    command: server /data
    ports:
      - "9000:9000"
    volumes:
      - minio_data:/data

  postgres:
    image: postgres:alpine
    environment:
      POSTGRES_DB: postgres
      POSTGRES_USER: myuser
      POSTGRES_PASSWORD: mypassword
    ports:
      - "5432:5432"
    volumes:
      - postgres_data:/var/lib/postgresql/data

volumes:
  minio_data:
  postgres_data:
```

In [6]:
!docker run -d \
    --name postgres \
    -e POSTGRES_DB=postgres \
    -e POSTGRES_USER=myuser \
    -e POSTGRES_PASSWORD=mypassword \
    -p 5432:5432 \
    -v postgres_data:/var/lib/postgresql/data \
    postgres:alpine

c580a2f2a48ce7612d90bee14fd827fd2c994605119d824cf1ce69178376d99d


Unable to find image 'postgres:alpine' locally
alpine: Pulling from library/postgres
8df61a580835: Download complete
661ff4d9561e: Download complete
c1c34308d9d2: Download complete
38f40ca4652e: Download complete
d73c6d20388d: Download complete
73cae0abb5d4: Download complete
14bfb625f63b: Download complete
db11692d92da: Download complete
742592b22bac: Download complete
Digest: sha256:a1b267d05ee39210d162185f52645687c7e63fbe25b8c58ccd7f81f0a7e2ad97
Status: Downloaded newer image for postgres:alpine


In [26]:
!docker ps -f name=postgres

CONTAINER ID   IMAGE             COMMAND                  CREATED      STATUS      PORTS                    NAMES
c580a2f2a48c   postgres:alpine   "docker-entrypoint.sâ€¦"   5 days ago   Up 5 days   0.0.0.0:5432->5432/tcp   postgres


In [27]:
!docker exec postgres psql -U myuser -d postgres -c "CREATE TABLE IF NOT EXISTS events (id SERIAL PRIMARY KEY, event_type VARCHAR(255), data JSON);"

CREATE TABLE


NOTICE:  relation "events" already exists, skipping


In [8]:
from pydantic import BaseModel
from flask import Flask, request
from minio import Minio
import psycopg2

# Pydantic configuration class for MinIO client
class MinioClientConfig(BaseModel):
    endpoint: str = '192.168.0.11:9000'  # Using container hostname
    access_key: str = 'minio'
    secret_key: str = 'minio123'

# Pydantic configuration class for PostgreSQL client
class PostgresClientConfig(BaseModel):
    host: str = 'localhost'  # Using container hostname
    port: int = 5432
    user: str = 'myuser'
    password: str = 'mypassword'
    database: str = 'postgres'

# Pydantic class for event structure
class Event(BaseModel):
    event_type: str
    data: dict
# Create an instance of MinioClientConfig
minio_config = MinioClientConfig()

# Initialize MinIO client using the instance
minio_client = Minio(minio_config.endpoint, 
                     access_key=minio_config.access_key, 
                     secret_key=minio_config.secret_key)

# Create an instance of PostgresClientConfig
postgres_config = PostgresClientConfig()

# Initialize PostgreSQL connection using the instance
pg_conn = psycopg2.connect(
    host=postgres_config.host,
    port=postgres_config.port,
    user=postgres_config.user,
    password=postgres_config.password,
    dbname=postgres_config.database)

# Flask app initialization
app = Flask(__name__)

# Route for handling events
@app.route('/event', methods=['POST'])
def handle_event():
    event_data = request.json
    event = Event(**event_data)
    process_event(event)
    return "Event processed"

# Function to process events
def process_event(event: Event):
    # Process event and log data in PostgreSQL
    try:
        with pg_conn.cursor() as cur:
            cur.execute("INSERT INTO events (event_type, data) VALUES (%s, %s)", 
                        (event.event_type, str(event.data)))
            pg_conn.commit()
    except Exception as e:
        print(f"Error processing event: {e}")

# Simulate an event for demonstration
simulated_event_data = {
    'event_type': 'file_uploaded',
    'data': {
        'file_name': 'example.txt',
        'file_size': '1024',
        'bucket_name': 'test-bucket'
    }
}

# Create Event instance with simulated data
simulated_event = Event(**simulated_event_data)

# Process the simulated event
process_event(simulated_event)

# Run the Flask app if this script is the main program
if __name__ == "__main__":
    app.run(debug=True)


Error processing event: relation "events" does not exist
LINE 1: INSERT INTO events (event_type, data) VALUES ('file_uploaded...
                    ^

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\david\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [14]:
from pydantic import BaseModel
from flask import Flask, request
from minio import Minio
import psycopg2
import json

# Pydantic configuration class for MinIO client
class MinioClientConfig(BaseModel):
    endpoint: str = '192.168.0.11:9000'  # Using container hostname
    access_key: str = 'minio'
    secret_key: str = 'minio123'

# Pydantic configuration class for PostgreSQL client
class PostgresClientConfig(BaseModel):
    host: str = 'localhost'  # Using container hostname
    port: int = 5432
    user: str = 'myuser'
    password: str = 'mypassword'
    database: str = 'postgres'

# Pydantic class for event structure
class Event(BaseModel):
    event_type: str
    data: dict

# Create an instance of MinioClientConfig
minio_config = MinioClientConfig()

# Initialize MinIO client using the instance
minio_client = Minio(minio_config.endpoint, 
                     access_key=minio_config.access_key, 
                     secret_key=minio_config.secret_key)

# Create an instance of PostgresClientConfig
postgres_config = PostgresClientConfig()

# Initialize PostgreSQL connection using the instance
pg_conn = psycopg2.connect(
    host=postgres_config.host,
    port=postgres_config.port,
    user=postgres_config.user,
    password=postgres_config.password,
    dbname=postgres_config.database)

def create_events_table():
    create_table_query = """
    CREATE TABLE IF NOT EXISTS events (
        id SERIAL PRIMARY KEY,
        event_type VARCHAR(255),
        data JSON
    );
    """
    try:
        with pg_conn.cursor() as cur:
            cur.execute(create_table_query)
            pg_conn.commit()
            print("Table 'events' created successfully.")
    except Exception as e:
        print(f"Error creating table: {e}")

# Flask app initialization
app = Flask(__name__)

# Route for handling events
@app.route('/event', methods=['POST'])
def handle_event():
    event_data = request.json
    event = Event(**event_data)
    process_event(event)
    return "Event processed"

# # Function to process events
# def process_event(event: Event):
#     # Process event and log data in PostgreSQL
#     try:
#         with pg_conn.cursor() as cur:
#             cur.execute("INSERT INTO events (event_type, data) VALUES (%s, %s)", 
#                         (event.event_type, str(event.data)))
#             pg_conn.commit()
#     except Exception as e:
#         print(f"Error processing event: {e}")

# Function to process events
def process_event(event: Event):
    # Serialize data to JSON
    json_data = json.dumps(event.data)

    # Process event and log data in PostgreSQL
    try:
        with pg_conn.cursor() as cur:
            cur.execute("INSERT INTO events (event_type, data) VALUES (%s, %s)", 
                        (event.event_type, json_data))
            pg_conn.commit()
    except Exception as e:
        pg_conn.rollback()  # Roll back the transaction in case of an error
        print(f"Error processing event: {e}")
        
# Simulate an event for demonstration
simulated_event_data = {
    'event_type': 'file_uploaded',
    'data': {
        'file_name': 'example.txt',
        'file_size': '1024',
        'bucket_name': 'test-bucket'
    }
}

# Create Event instance with simulated data
simulated_event = Event(**simulated_event_data)

# Process the simulated event
process_event(simulated_event)

# Run the Flask app if this script is the main program
if __name__ == "__main__":
    create_events_table()
    app.run(debug=True, port=5000)

Table 'events' created successfully.
 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\david\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [19]:
from pydantic import BaseModel
from flask import Flask, request
from minio import Minio
import psycopg2
import json

# Pydantic configuration class for MinIO client
class MinioClientConfig(BaseModel):
    endpoint: str = '192.168.0.11:9000'  # Using container hostname
    access_key: str = 'minio'
    secret_key: str = 'minio123'

# Pydantic configuration class for PostgreSQL client
class PostgresClientConfig(BaseModel):
    host: str = 'localhost'  # Using container hostname
    port: int = 5432
    user: str = 'myuser'
    password: str = 'mypassword'
    database: str = 'postgres'

# Pydantic class for event structure
class Event(BaseModel):
    event_type: str
    data: dict

# Create an instance of MinioClientConfig
minio_config = MinioClientConfig()

# Initialize MinIO client using the instance
minio_client = Minio(minio_config.endpoint, 
                     access_key=minio_config.access_key, 
                     secret_key=minio_config.secret_key)

# Create an instance of PostgresClientConfig
postgres_config = PostgresClientConfig()

# Initialize PostgreSQL connection using the instance
pg_conn = psycopg2.connect(
    host=postgres_config.host,
    port=postgres_config.port,
    user=postgres_config.user,
    password=postgres_config.password,
    dbname=postgres_config.database)

def create_events_table():
    create_table_query = """
    CREATE TABLE IF NOT EXISTS events (
        id SERIAL PRIMARY KEY,
        event_type VARCHAR(255),
        data JSON
    );
    """
    try:
        with pg_conn.cursor() as cur:
            cur.execute(create_table_query)
            pg_conn.commit()
            print("Table 'events' created successfully.")
    except Exception as e:
        print(f"Error creating table: {e}")

# Flask app initialization
app = Flask(__name__)

# Route for handling events
@app.route('/event', methods=['POST'])
def handle_event():
    event_data = request.json
    event = Event(**event_data)
    process_event(event)
    return "Event processed"

# Function to process events
def process_event(event: Event):
    # Serialize data to JSON
    json_data = json.dumps(event.data)

    # Process event and log data in PostgreSQL
    try:
        with pg_conn.cursor() as cur:
            cur.execute("INSERT INTO events (event_type, data) VALUES (%s, %s)", 
                        (event.event_type, json_data))
            pg_conn.commit()
    except Exception as e:
        pg_conn.rollback()  # Roll back the transaction in case of an error
        print(f"Error processing event: {e}")
        
# Simulate an event for demonstration
simulated_event_data = {
    'event_type': 'file_uploaded',
    'data': {
        'file_name': 'example.txt',
        'file_size': '1024',
        'bucket_name': 'test-bucket'
    }
}

# Create Event instance with simulated data
simulated_event = Event(**simulated_event_data)

# Process the simulated event
process_event(simulated_event)

# Run the Flask app if this script is the main program
if __name__ == "__main__":
    create_events_table()
    app.run(debug=True, port=5000)

Table 'events' created successfully.
 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\david\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [24]:

from pydantic import BaseModel
from flask import Flask, request
from minio import Minio
import psycopg2
import json

# Pydantic configuration class for MinIO client
class MinioClientConfig(BaseModel):
    endpoint: str = '192.168.0.11:9000'  # Using container hostname
    access_key: str = 'minio'
    secret_key: str = 'minio123'

# Pydantic configuration class for PostgreSQL client
class PostgresClientConfig(BaseModel):
    host: str = 'localhost'  # Using container hostname
    port: int = 5432
    user: str = 'myuser'
    password: str = 'mypassword'
    database: str = 'postgres'

# Pydantic class for event structure
class Event(BaseModel):
    event_type: str
    data: dict

# Create an instance of MinioClientConfig
minio_config = MinioClientConfig()

# Initialize MinIO client using the instance
minio_client = Minio(minio_config.endpoint, 
                     access_key=minio_config.access_key, 
                     secret_key=minio_config.secret_key)

# Create an instance of PostgresClientConfig
postgres_config = PostgresClientConfig()

# Initialize PostgreSQL connection using the instance
pg_conn = psycopg2.connect(
    host=postgres_config.host,
    port=postgres_config.port,
    user=postgres_config.user,
    password=postgres_config.password,
    dbname=postgres_config.database)

# def create_events_table():
#     create_table_query = """
#     CREATE TABLE IF NOT EXISTS events (
#         id SERIAL PRIMARY KEY,
#         event_type VARCHAR(255),
#         data JSON
#     );
#     """
#     try:
#         with pg_conn.cursor() as cur:
#             cur.execute(create_table_query)
#             pg_conn.commit()
#             print("Table 'events' has been successfully.")
#     except Exception as e:
#         print(f"Error creating table: {e}")

# Flask app initialization
app = Flask(__name__)

# Route for handling events
@app.route('/event', methods=['POST'])
def handle_event():
    event_data = request.json
    event = Event(**event_data)
    process_event(event)
    return "Event processed"

# Function to process events
def process_event(event: Event):
    # Serialize data to JSON
    json_data = json.dumps(event.data)

    # Process event and log data in PostgreSQL
    try:
        with pg_conn.cursor() as cur:
            cur.execute("INSERT INTO events (event_type, data) VALUES (%s, %s)", 
                        (event.event_type, json_data))
            pg_conn.commit()
            print("Event Notification for 'simulated_event_data' has been inserted successfully into 'Events' Table.")
    except Exception as e:
        pg_conn.rollback()  # Roll back the transaction in case of an error
        print(f"Error processing event: {e}")
        
# Simulate an event for demonstration
simulated_event_data = {
    'event_type': 'file_uploaded',
    'data': {
        'file_name': 'example.txt',
        'file_size': '1024',
        'bucket_name': 'test-bucket'
    }
}

# Create Event instance with simulated data
simulated_event = Event(**simulated_event_data)

# Process the simulated event
process_event(simulated_event)

# Run the Flask app if this script is the main program
if __name__ == "__main__":
    create_events_table()
    app.run(debug=True, port=5000)

Event Notification for 'simulated_event_data' has been inserted successfully.
Table 'events' has been successfully.
 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\david\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [25]:
!docker exec postgres psql -U myuser -d postgres -c "SELECT * FROM events;"

 id |  event_type   |                                      data                                       
----+---------------+---------------------------------------------------------------------------------
  1 | file_uploaded | {"file_name": "example.txt", "file_size": "1024", "bucket_name": "test-bucket"}
  2 | file_uploaded | {"file_name": "example.txt", "file_size": "1024", "bucket_name": "test-bucket"}
  3 | file_uploaded | {"file_name": "example.txt", "file_size": "1024", "bucket_name": "test-bucket"}
  4 | file_uploaded | {"file_name": "example.txt", "file_size": "1024", "bucket_name": "test-bucket"}
  5 | file_uploaded | {"file_name": "example.txt", "file_size": "1024", "bucket_name": "test-bucket"}
  6 | file_uploaded | {"file_name": "example.txt", "file_size": "1024", "bucket_name": "test-bucket"}
  7 | file_uploaded | {"file_name": "example.txt", "file_size": "1024", "bucket_name": "test-bucket"}
(7 rows)

